In [1]:
import math,csv, random, collections
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, HTML
import scipy.sparse as sparse
from gurobipy import *
import h5py

from collections import Counter
import imp, time
from numpy import linalg as LA

pd.set_option('display.max_rows', 6)
        
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'



ImportError: No module named gurobipy

ERROR:tornado.application:Exception in callback <functools.partial object at 0x7f3038c8b3c0>
Traceback (most recent call last):
  File "/home/govinda/anaconda/envs/bandits/lib/python2.7/site-packages/tornado/ioloop.py", line 600, in _run_callback
    ret = callback()
  File "/home/govinda/anaconda/envs/bandits/lib/python2.7/site-packages/tornado/stack_context.py", line 275, in null_wrapper
    return fn(*args, **kwargs)
  File "/home/govinda/anaconda/envs/bandits/lib/python2.7/site-packages/ipykernel/kernelbase.py", line 234, in enter_eventloop
    self.eventloop(self)
  File "/home/govinda/anaconda/envs/bandits/lib/python2.7/site-packages/ipykernel/eventloops.py", line 163, in loop_tk
    kernel.timer = Timer(doi)
  File "/home/govinda/anaconda/envs/bandits/lib/python2.7/site-packages/ipykernel/eventloops.py", line 151, in __init__
    self.app = Tk()
  File "/home/govinda/anaconda/envs/bandits/lib/python2.7/lib-tk/Tkinter.py", line 1815, in __init__
    self.tk = _tkinter.create(scre